In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gabriel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("dataset.csv")
train, test = train_test_split(df, test_size=0.2)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
train["sentence"] = train["sentence"].str.strip().str.lower()
test["sentence"] = test["sentence"].str.strip().str.lower()

In [3]:
train["category"] = train["category"].str.split(",")
test["category"] = test["category"].str.split(",")

mlb = MultiLabelBinarizer()

one_hot_encoded_train = mlb.fit_transform(train['category'])
one_hot_train_df = pd.DataFrame(one_hot_encoded_train, columns=mlb.classes_)
train = pd.concat([train, one_hot_train_df], axis=1).drop('category', axis=1)

one_hot_encoded_test = mlb.transform(test['category'])
one_hot_test_df = pd.DataFrame(one_hot_encoded_test, columns=mlb.classes_)
test = pd.concat([test, one_hot_test_df], axis=1).drop('category', axis=1)

In [14]:
cat = ['educação', 'finanças', 'indústrias', 'orgão público',
       'varejo']

In [15]:
stop_words_pt = stopwords.words('portuguese')
vec = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9,
                      strip_accents='unicode', use_idf=1,
                      smooth_idf=1, sublinear_tf=1 )
X_train_matrix = vec.fit_transform(train["sentence"])
X_test_matrix = vec.transform(test["sentence"])

y_train = train[cat]
y_test = test[cat]

/home/gabriel/.pyenv/versions/3.8.0/envs/airflow_dags/lib/python3.8/site-packages/sklearn/utils/_param_validation.py:594: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


In [ ]:
# create a function to add features
def add_feature(X, feature_to_add):
    '''
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    '''
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(C=12.0)

for label in cat:
    print(f'... Processing {label}')
    y = train[label]
    # train the model using X_dtm & y
    logreg.fit(X_dtm,y)
    # compute the training accuracy
    y_pred_X = logreg.predict(X_dtm)
    print(f'Training Accuracy is {accuracy_score(y, y_pred_X)}')
    # make predictions from test_X
    test_y = logreg.predict(test_X_dtm)
    test_y_prob = logreg.predict_proba(test_X_dtm)[:,1]
    submission_chains[label] = test_y_prob
    # chain current label to X_dtm
    X_dtm = add_feature(X_dtm, y)
    print(f'Shape of X_dtm is now {X_dtm.shape}')
    # chain current label predictions to test_X_dtm
    test_X_dtm = add_feature(test_X_dtm, test_y)
    print(f'Shape of test_X_dtm is now {test_X_dtm.shape}')



In [16]:
X_train

NameError: name 'X_train' is not defined

In [ ]:
from sklearn.tree import DecisionTreeClassifier
kf = KFold(n_splits=5)
dict_acc = dict(zip(cat, [[] for _ in range(len(cat))]))
model_dict = {}
for i, (train_index, test_index) in enumerate(kf.split(X_train_matrix)):
    X_tr, X_te = X_train_matrix[train_index], X_train_matrix[test_index]
    y_tr, y_te = y_train.loc[train_index], y_train.loc[test_index]
    print(f"Training in Fold: {i}")
    for label in cat:
        # model = LogisticRegression(C=15)
        model = DecisionTreeClassifier()
        print(f'Processing {label}')
        model.fit(X_tr, y_tr[label])
        y_pred = model.predict(X_te)
        score = accuracy_score(y_te[label], y_pred)
        print(f'Training accuracy is {score}')
        test_y_prob = model.predict_proba(X_te)[:,1]
        model_dict[label] = model
        dict_acc[label].append(score)
        if score > np.max(dict_acc[label]):
            model_dict[label] = model
    print("*" * 50)

In [5]:
trn_term_doc

<416x336 sparse matrix of type '<class 'numpy.float64'>'
	with 2362 stored elements in Compressed Sparse Row format>